In [1]:
# ——— Environment and imports ———

!pip install xgboost scikit-learn
!pip install /kaggle/input/datasets/kami1976/biopython-cp312/biopython-1.86-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl

import numpy as np
import pandas as pd
import joblib
import os, sys, time
import warnings
import math

# Bio imports
from Bio.Align import PairwiseAligner

# ML imports

from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_squared_error
import xgboost as xgb

Processing /kaggle/input/datasets/kami1976/biopython-cp312/biopython-1.86-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl


In [2]:
# ——— Load CSVs from competition input ———

warnings.filterwarnings('ignore')

DATA_PATH = '/kaggle/input/competitions/stanford-rna-3d-folding-2'
train_seqs = pd.read_csv(DATA_PATH + '/train_sequences.csv')
test_seqs = pd.read_csv(DATA_PATH + '/test_sequences.csv')
train_labels = pd.read_csv(DATA_PATH + '/train_labels.csv')

sys.path.append(os.path.join(DATA_PATH, "/extra"))

In [3]:
# --- Robust import for Kaggle's extra/parse_fasta_py.py ---
try:
    import typing as _typing
    import builtins as _builtins

    # Make these names available during module import-time annotation evaluation
    _builtins.Dict  = getattr(_typing, "Dict")
    _builtins.Tuple = getattr(_typing, "Tuple")
    _builtins.List  = getattr(_typing, "List")

    from parse_fasta_py import parse_fasta as _parse_fasta_raw

    # Normalize output to: {chain_id: sequence_string}
    def parse_fasta(fasta_content: str):
        d = _parse_fasta_raw(fasta_content)
        out = {}
        for k, v in d.items():
            # some variants return (sequence, headers/lines) or similar
            out[k] = v[0] if isinstance(v, tuple) else v
        return out

except Exception:
    # Fallback FASTA parser: {chain_id: sequence_string}
    def parse_fasta(fasta_content: str):
        out = {}
        cur = None
        seq_parts = []
        for line in str(fasta_content).splitlines():
            line = line.strip()
            if not line:
                continue
            if line.startswith(">"):
                if cur is not None:
                    out[cur] = "".join(seq_parts)
                header = line[1:]
                # First token is usually chain id in this dataset
                cur = header.split()[0]
                seq_parts = []
            else:
                seq_parts.append(line.replace(" ", ""))
        if cur is not None:
            out[cur] = "".join(seq_parts)
        return out

def parse_stoichiometry(stoich: str):
    if pd.isna(stoich) or str(stoich).strip() == "":
        return []
    out = []
    for part in str(stoich).split(';'):
        ch, cnt = part.split(':')
        out.append((ch.strip(), int(cnt)))
    return out

def get_chain_segments(row):
    """
    Returns list of (start,end) segments in row['sequence'] corresponding to chain copies in stoichiometry order.
    Falls back to single segment if parsing fails.
    """
    seq = row['sequence']
    stoich = row.get('stoichiometry', '')
    all_seq = row.get('all_sequences', '')

    if pd.isna(stoich) or pd.isna(all_seq) or str(stoich).strip()=="" or str(all_seq).strip()=="":
        return [(0, len(seq))]

    try:
        chain_dict = parse_fasta(all_seq)  # dict: chain_id -> sequence
        order = parse_stoichiometry(stoich)
        segs = []
        pos = 0
        for ch, cnt in order:
            base = chain_dict.get(ch)
            if base is None:
                return [(0, len(seq))]
            for _ in range(cnt):
                L = len(base)
                segs.append((pos, pos + L))
                pos += L
        if pos != len(seq):
            return [(0, len(seq))]
        return segs
    except Exception:
        return [(0, len(seq))]

# Build maps for train and test for quick use
def build_segments_map(df):
    seg_map = {}
    stoich_map = {}
    for _, r in df.iterrows():
        tid = r['target_id']
        seg_map[tid] = get_chain_segments(r)
        stoich_map[tid] = str(r.get('stoichiometry', '') if not pd.isna(r.get('stoichiometry', '')) else '')
    return seg_map, stoich_map

train_segs_map, train_stoich_map = build_segments_map(train_seqs)
test_segs_map,  test_stoich_map  = build_segments_map(test_seqs)

In [4]:
# ——— Process train_labels into dict of coords per target_id ———

def process_labels(labels_df):
    coords_dict = {}
    # Faster + safer prefix extraction
    prefixes = labels_df['ID'].str.rsplit('_', n=1).str[0]
    for id_prefix, group in labels_df.groupby(prefixes):
        coords_dict[id_prefix] = group.sort_values('resid')[['x_1', 'y_1', 'z_1']].values
    return coords_dict

train_coords_dict = process_labels(train_labels)

In [5]:
# ——— Aligner: Biopython ———

# Scoring parameters
MATCH_SCORE = 2.0
MISMATCH_SCORE = -1.6
# Residue numbering must match
GAP_OPEN = -8.0
GAP_EXTEND = -0.4

# Create Aligner
aligner = PairwiseAligner()
aligner.mode = 'global'

# Define scores
aligner.match_score = MATCH_SCORE
aligner.mismatch_score = MISMATCH_SCORE
aligner.open_gap_score = GAP_OPEN
aligner.extend_gap_score = GAP_EXTEND

# Penalize terminal gaps
aligner.query_left_open_gap_score  = GAP_OPEN
aligner.query_left_extend_gap_score = GAP_EXTEND
aligner.query_right_open_gap_score = GAP_OPEN
aligner.query_right_extend_gap_score = GAP_EXTEND
aligner.target_left_open_gap_score = GAP_OPEN
aligner.target_left_extend_gap_score = GAP_EXTEND
aligner.target_right_open_gap_score = GAP_OPEN
aligner.target_right_extend_gap_score = GAP_EXTEND

In [6]:
def find_similar_sequences(query_seq, train_seqs_df, train_coords_dict, top_n=5):
    similar_seqs = []
    
    # Pre-filter: Iterate only valid targets
    # Note: aligner.score is much faster than generating full alignments
    for _, row in train_seqs_df.iterrows():
        target_id, train_seq = row['target_id'], row['sequence']
        if target_id not in train_coords_dict: continue
        
        # Length filter (keep your original logic)
        if abs(len(train_seq) - len(query_seq)) / max(len(train_seq), len(query_seq)) > 0.3: continue
        
        # FAST SCORE: Calculates score without traceback overhead
        raw_score = aligner.score(query_seq, train_seq)
        
        normalized_score = raw_score / (2 * min(len(query_seq), len(train_seq)))
        similar_seqs.append((target_id, train_seq, normalized_score, train_coords_dict[target_id]))
    
    similar_seqs.sort(key=lambda x: x[2], reverse=True)
    return similar_seqs[:top_n]

def adapt_template_to_query(query_seq, template_seq, template_coords):
    # Generate the alignment object
    # aligner.align returns an iterator; we take the first optimal alignment
    alignment = next(iter(aligner.align(query_seq, template_seq)))
    
    new_coords = np.full((len(query_seq), 3), np.nan)
    
    # VECTORIZED MAPPING:
    # alignment.aligned returns lists of (start, end) tuples for matched segments.
    # This avoids the slow python loop "for char_q, char_t in zip..."
    for (q_start, q_end), (t_start, t_end) in zip(*alignment.aligned):
        # Map the coordinate chunk directly
        t_chunk = template_coords[t_start:t_end]
        
        # Safety check to ensure shapes match (handles edge cases)
        if len(t_chunk) == (q_end - q_start):
            new_coords[q_start:q_end] = t_chunk

    # --- Interpolation Logic (Unchanged) ---
    for i in range(len(new_coords)):
        if np.isnan(new_coords[i, 0]):
            prev_v = next((j for j in range(i-1, -1, -1) if not np.isnan(new_coords[j, 0])), -1)
            next_v = next((j for j in range(i+1, len(new_coords)) if not np.isnan(new_coords[j, 0])), -1)
            if prev_v >= 0 and next_v >= 0:
                w = (i - prev_v) / (next_v - prev_v)
                new_coords[i] = (1-w)*new_coords[prev_v] + w*new_coords[next_v]
            elif prev_v >= 0: new_coords[i] = new_coords[prev_v] + [3, 0, 0]
            elif next_v >= 0: new_coords[i] = new_coords[next_v] + [3, 0, 0]
            else: new_coords[i] = [i*3, 0, 0]
            
    return np.nan_to_num(new_coords)

def adaptive_rna_constraints(coordinates, target_id, confidence=1.0, passes=2):
    """
    Evaluation-driven constraints:
    - US-align is show-only rigid body => internal geometry errors are fatal
    - apply within each chain segment (no fake bond across chain breaks)
    """
    coords = coordinates.copy()
    segments = test_segs_map.get(target_id, [(0, len(coords))])

    # stronger corrections when confidence is low
    strength = 0.80 * (1.0 - min(confidence, 0.98))
    strength = max(strength, 0.02)

    for _ in range(passes):
        for (s, e) in segments:
            X = coords[s:e]
            L = e - s
            if L < 3:
                coords[s:e] = X
                continue

            # (1) bond i,i+1 to ~5.95Å (vectorized, symmetric)
            d = X[1:] - X[:-1]
            dist = np.linalg.norm(d, axis=1) + 1e-5
            target = 5.95
            scale = (target - dist) / dist
            adj = (d * scale[:, None]) * (0.22 * strength)
            X[:-1] -= adj
            X[1:]  += adj

            # (2) soft i,i+2 to ~10.2Å (vectorized, symmetric)
            d2 = X[2:] - X[:-2]
            dist2 = np.linalg.norm(d2, axis=1) + 1e-6
            target2 = 10.2
            scale2 = (target2 - dist2) / dist2
            adj2 = (d2 * scale2[:, None]) * (0.10 * strength)
            X[:-2] -= adj2
            X[2:]  += adj2

            # (3) Laplacian smoothing (removes kinks US-align cannot fix)
            lap = 0.5 * (X[:-2] + X[2:]) - X[1:-1]
            X[1:-1] += (0.06 * strength) * lap

            # (4) light self-avoidance (prevents steric collapse)
            if L >= 25:
                k = min(L, 160) if L > 220 else L
                if k < L:
                    idx = np.linspace(0, L - 1, k).astype(int)
                else:
                    idx = np.arange(L)

                P = X[idx]
                diff = P[:, None, :] - P[None, :, :]
                distm = np.linalg.norm(diff, axis=2) + 1e-6
                sep = np.abs(idx[:, None] - idx[None, :])

                mask = (sep > 2) & (distm < 3.3)
                if np.any(mask):
                    force = (3.3 - distm) / distm
                    vec = (diff * force[:, :, None] * mask[:, :, None]).sum(axis=1)
                    X[idx] += (0.015 * strength) * vec

            coords[s:e] = X

    return coords

def _rotmat(axis, ang):
    axis = np.asarray(axis, float)
    axis = axis / (np.linalg.norm(axis) + 1e-12)
    x, y, z = axis
    c, s = np.cos(ang), np.sin(ang)
    C = 1.0 - c
    return np.array([
        [c + x*x*C,     x*y*C - z*s, x*z*C + y*s],
        [y*x*C + z*s,   c + y*y*C,   y*z*C - x*s],
        [z*x*C - y*s,   z*y*C + x*s, c + z*z*C]
    ], dtype=float)

def apply_hinge(coords, seg, rng, max_angle_deg=25):
    s, e = seg
    L = e - s
    if L < 30:
        return coords
    pivot = s + int(rng.integers(10, L - 10))
    axis = rng.normal(size=3)
    ang = np.deg2rad(float(rng.uniform(-max_angle_deg, max_angle_deg)))
    R = _rotmat(axis, ang)
    X = coords.copy()
    p0 = X[pivot].copy()
    X[pivot+1:e] = (X[pivot+1:e] - p0) @ R.T + p0
    return X

def jitter_chains(coords, segments, rng, max_angle_deg=12, max_trans=1.5):
    X = coords.copy()
    global_center = X.mean(axis=0, keepdims=True)
    for (s, e) in segments:
        axis = rng.normal(size=3)
        ang = np.deg2rad(float(rng.uniform(-max_angle_deg, max_angle_deg)))
        R = _rotmat(axis, ang)
        shift = rng.normal(size=3)
        shift = shift / (np.linalg.norm(shift) + 1e-10) * float(rng.uniform(0.0, max_trans))
        c = X[s:e].mean(axis=0, keepdims=True)
        X[s:e] = (X[s:e] - c) @ R.T + c + shift
    # recenter
    X -= X.mean(axis=0, keepdims=True) - global_center
    return X

def smooth_wiggle(coords, segments, rng, amp=0.8):
    X = coords.copy()
    for (s, e) in segments:
        L = e - s
        if L < 20:
            continue
        n_ctrl = 6
        ctrl_x = np.linspace(0, L - 1, n_ctrl)
        ctrl_disp = rng.normal(0, amp, size=(n_ctrl, 3))
        t = np.arange(L)
        disp = np.vstack([np.interp(t, ctrl_x, ctrl_disp[:, k]) for k in range(3)]).T
        X[s:e] += disp
    return X

def predict_rna_structures(row, train_seqs_df, train_coords_dict, n_predictions=5):
    tid = row['target_id']
    seq = row['sequence']

    # Data constraint: should already be canonical A/C/G/U
    assert set(seq).issubset(set("ACGU")), f"Non-ACGU in {tid}; do not remap here."

    segments = test_segs_map.get(tid, [(0, len(seq))])

    # Grab a larger candidate pool, then sample for diversity (best-of-5)
    cands = find_similar_sequences(query_seq=seq, train_seqs_df=train_seqs_df, train_coords_dict=train_coords_dict, top_n=40)
    assert all(len(c[3]) == len(c[1]) for c in cands), "Template coords/seq length mismatch"
    predictions = []
    used = set()

    for i in range(n_predictions):
        seed = (abs(hash(tid)) + i * 10005) % (2**32)
        rng = np.random.default_rng(seed)

        if not cands:
            # hard fallback (straight line per chain)
            coords = np.zeros((len(seq), 3), dtype=float)
            for (s, e) in segments:
                for j in range(s+1, e):
                    coords[j] = coords[j-1] + [5.95, 0, 0]
            predictions.append(coords)
            continue

        # Choose template:
        # i=0 => best template; others => sample among top-K with weights, avoid duplicates
        if i == 0:
            t_id, t_seq, sim, t_coords = cands[0]
        else:
            K = min(12, len(cands))
            sims = np.array([cands[k][2] for k in range(K)], float)
            w = np.exp((sims - sims.max()) / 0.10)
            # penalize already used templates
            for k in range(K):
                if cands[k][0] in used:
                    w[k] *= 0.10
            w = w / (w.sum() + 1e-10)
            k = int(rng.choice(np.arange(K), p=w))
            t_id, t_seq, sim, t_coords = cands[k]

        used.add(t_id)

        # Transfer coords with diagonal-guard mapping (no sliding)
        adapted = adapt_template_to_query(query_seq=seq, template_seq=t_seq, template_coords=t_coords)

        # Diversity transforms (then re-refine constraints)
        if i == 0:
            X = adapted
        elif i == 1:
            # mild noise
            X = adapted + rng.normal(0, max(0.01, (0.40 - sim) * 0.06), adapted.shape)
        elif i == 2:
            # hinge within the longest chain
            longest = max(segments, key=lambda se: se[1] - se[0])
            X = apply_hinge(adapted, longest, rng, max_angle_deg=22)
        elif i == 3:
            # inter-chain jitter (small, safe)
            X = jitter_chains(adapted, segments, rng, max_angle_deg=10, max_trans=1.0)
        else:
            # smooth low-frequency deformation
            X = smooth_wiggle(adapted, segments, rng, amp=0.8)

        refined = adaptive_rna_constraints(X, tid, confidence=sim, passes=2)
        predictions.append(refined)

    return predictions

In [7]:
# --- Provide window one-hot encoding and k-mer helpers used for prefiltering and features ---

NT_ORDER = ['A','C','G','U']
nt_to_onehot = {nt: np.array([1 if nt == x else 0 for x in NT_ORDER]) for nt in NT_ORDER}

def seq_window_onehot(seq, idx, W=2):
    vecs = []
    L = len(seq)
    for k in range(idx - W, idx + W + 1):
        if 0 <= k < L:
            v = nt_to_onehot.get(seq[k], np.zeros(4))
        else:
            v = np.zeros(4)
        vecs.append(v)
    return np.concatenate(vecs)  # (2W+1)*4

def kmer_set(s, k=3):
    if len(s) < k:
        return set([s])
    return set(s[i:i+k] for i in range(len(s)-k+1))

def jaccard(a, b):
    if not a or not b:
        return 0.0
    ia = a & b
    return len(ia) / len(a | b)

In [8]:
# --- build residual dataset ---

def build_residual_dataset(train_seqs_df, train_coords_dict, 
                           top_template_k=5, window=2, max_targets=None,
                           max_pos_per_target=None, sample_positions=False,
                           kmer_prefilter_top=None):
    """
    Builds features and targets (residuals) to train models.
    Parameters:
      - max_targets: limit on the number of targets to speed up testing.
      - max_pos_per_target: limit on sampled positions per target (useful for testing).
      - sample_positions: if True, randomly samples positions; if False, uses all up to the limit.
      - kmer_prefilter_top: if int, pre-filters templates by top-N Jaccard k-mer before using the aligner (speeds up).
    Returns (feats_df, targets_array).
    """

    rows = []
    targets = []
    nt_window_size = window

    # precompute kmer sets
    kmer_sets = {}
    if kmer_prefilter_top:
        for _, r in train_seqs_df.iterrows():
            kmer_sets[r['target_id']] = kmer_set(r['sequence'], k=3)

    ids = list(train_seqs_df['target_id'])
    if max_targets:
        ids = ids[:max_targets]

    for tid in ids:
        row = train_seqs_df[train_seqs_df['target_id'] == tid].iloc[0]
        seq = row['sequence']
        L = len(seq)
        if tid not in train_coords_dict:
            continue
        true_coords = train_coords_dict[tid]  # shape (L,3) expected

        # skip if true coords have NaN everywhere or shape mismatch
        if true_coords is None or true_coords.shape[0] != L:
            continue

        # choose template candidates
        if kmer_prefilter_top:
            q_k = kmer_set(seq, k=3)
            # compute jaccard to all targets quickly
            scores = []
            for _, r2 in train_seqs_df.iterrows():
                tid2 = r2['target_id']
                if tid2 not in train_coords_dict or tid2 == tid:
                    continue
                j = jaccard(q_k, kmer_sets[tid2])
                scores.append((tid2, j))
            scores.sort(key=lambda x: x[1], reverse=True)
            cand_ids = [s[0] for s in scores[: max(200, top_template_k*10) ]]  # top candidates
            # build a filtered DataFrame of candidates
            cand_df = train_seqs_df[train_seqs_df['target_id'].isin(cand_ids)]
            cands = find_similar_sequences(seq, cand_df, train_coords_dict, top_n=top_template_k)
        else:
            cands = find_similar_sequences(seq, train_seqs_df, train_coords_dict, top_n=top_template_k)

        if not cands:
            continue

        t_id, t_seq, sim, t_coords = cands[0]
        adapted = adapt_template_to_query(seq, t_seq, t_coords)

        # build list of positions to include
        pos_indices = list(range(L))
        if max_pos_per_target:
            if sample_positions:
                rng = np.random.default_rng(abs(hash(tid)) % (2**32))
                pos_indices = rng.choice(np.arange(L), size=min(max_pos_per_target, L), replace=False).tolist()
            else:
                pos_indices = list(range(min(L, max_pos_per_target)))

        # iterate positions
        for i in pos_indices:
            true_pt = true_coords[i]
            adapt_pt = adapted[i]
            # filter if any NaN or infinite
            if not np.isfinite(true_pt).all():
                continue
            if not np.isfinite(adapt_pt).all():
                continue
            # compute features
            feat = {}
            feat['target_id'] = tid
            feat['pos'] = i
            feat['pos_norm'] = i / max(1, L-1)
            feat['seq_len'] = L
            feat['sim_score'] = sim
            feat['ax'], feat['ay'], feat['az'] = adapt_pt
            # window one-hot
            win = seq_window_onehot(seq, i, W=nt_window_size)
            for j, val in enumerate(win):
                feat[f'w_{j}'] = float(val)
            rows.append(feat)
            # target residual
            rx, ry, rz = true_pt - adapt_pt
            targets.append((float(rx), float(ry), float(rz)))

    if not rows:
        # nothing built
        return pd.DataFrame(), np.zeros((0,3), dtype=float)

    feats_df = pd.DataFrame(rows)
    targets_arr = np.array(targets, dtype=float)
    # Ensure no rows with NaN remain
    finite_mask = np.isfinite(targets_arr).all(axis=1)
    feats_df = feats_df.loc[finite_mask].reset_index(drop=True)
    targets_arr = targets_arr[finite_mask]

    return feats_df, targets_arr

In [9]:
# --- Quick build for development and inspect coverage ---

feats, targets = build_residual_dataset(train_seqs, train_coords_dict,
                                       top_template_k=5, window=2,
                                       max_targets=200, max_pos_per_target=100,
                                       sample_positions=True, kmer_prefilter_top=200)

if feats.shape[0] == 0:
    raise RuntimeError("No samples built - check train_coords_dict and functions adapt_template_to_query/find_similar_sequences.")

# define feature columns (all except identifiers)
feature_cols = [c for c in feats.columns if c not in ['target_id','pos']]

X = feats[feature_cols].values
y = targets  # (N,3)
groups = feats['target_id'].values

# clean NaNs / infinites just for safety
if y.ndim == 1:
    mask_y = np.isfinite(y)
else:
    mask_y = np.isfinite(y).all(axis=1)
mask_good = np.isfinite(X).all(axis=1) & mask_y

X = X[mask_good]
y = y[mask_good]
groups = groups[mask_good]

print("Training samples:", X.shape[0])

Training samples: 7230


In [10]:
# --- Training + Cross-validation ---

print("Samples:", X.shape, "Targets:", y.shape)
unique_targets = len(np.unique(groups))
print("Unique train targets:", unique_targets)

# Hyperparameters
rf_params = dict(n_estimators=120, max_depth=18, n_jobs=-1, random_state=42)
xgb_base = xgb.XGBRegressor(n_estimators=200, max_depth=6, tree_method='hist', verbosity=0, random_state=42)

# Cross-validation by target
n_splits = min(5, max(2, unique_targets))
gkf = GroupKFold(n_splits=n_splits)

rf_fold_scores = []
xgb_fold_scores = []
ens_fold_scores = []

fold = 0
for train_idx, val_idx in gkf.split(X, y, groups):
    fold += 1
    Xtr, Xval = X[train_idx], X[val_idx]
    ytr, yval = y[train_idx], y[val_idx]

    # RF multi-output
    rf = RandomForestRegressor(**rf_params)
    rf.fit(Xtr, ytr)

    # XGB multi-output (wrapper)
    xgb_multi = MultiOutputRegressor(xgb_base, n_jobs=3)
    xgb_multi.fit(Xtr, ytr)

    # Prediction and evaluation
    p_rf = rf.predict(Xval)
    p_xg = xgb_multi.predict(Xval)
    p_ens = (p_rf + p_xg) / 2.0

    rmse_rf = math.sqrt(mean_squared_error(yval, p_rf))
    rmse_xg = math.sqrt(mean_squared_error(yval, p_xg))
    rmse_ens = math.sqrt(mean_squared_error(yval, p_ens))

    print(f"Fold {fold}/{n_splits} -> RF RMSE: {rmse_rf:.4f} | XGB RMSE: {rmse_xg:.4f} | ENS RMSE: {rmse_ens:.4f}")

    rf_fold_scores.append(rmse_rf)
    xgb_fold_scores.append(rmse_xg)
    ens_fold_scores.append(rmse_ens)

print("CV mean RMSE -> RF:", np.mean(rf_fold_scores), "XGB:", np.mean(xgb_fold_scores), "ENS:", np.mean(ens_fold_scores))

# Final training on the whole dataset
print("Training final models on the entire dataset (may take a while)...")
rf_final = RandomForestRegressor(**rf_params)
rf_final.fit(X, y)

xgb_final = MultiOutputRegressor(xgb_base, n_jobs=3)
xgb_final.fit(X, y)

# Save models and metadata
joblib.dump(rf_final, 'rf_final.joblib')
joblib.dump(xgb_final, 'xgb_final.joblib')

# Save feature_cols and metadata (important for inference)
feature_cols = sorted(feature_cols)
joblib.dump(feature_cols, 'feature_cols.joblib')
metadata = {'window': 2, 'feature_cols_len': len(feature_cols)}  # adjust 'window' if using another
joblib.dump(metadata, 'model_metadata.joblib')

print("Models and artifacts saved: rf_final.joblib, xgb_final.joblib, feature_cols.joblib, model_metadata.joblib")

Samples: (7230, 26) Targets: (7230, 3)
Unique train targets: 196
Fold 1/5 -> RF RMSE: 25.1005 | XGB RMSE: 20.8089 | ENS RMSE: 21.7658
Fold 2/5 -> RF RMSE: 29.0151 | XGB RMSE: 30.3002 | ENS RMSE: 28.9025
Fold 3/5 -> RF RMSE: 58.3969 | XGB RMSE: 56.3454 | ENS RMSE: 55.8658
Fold 4/5 -> RF RMSE: 38.1238 | XGB RMSE: 59.5341 | ENS RMSE: 46.9830
Fold 5/5 -> RF RMSE: 30.9320 | XGB RMSE: 37.4067 | ENS RMSE: 32.1382
CV mean RMSE -> RF: 36.31364560229808 XGB: 40.87905199766221 ENS: 37.1310764759451
Training final models on the entire dataset (may take a while)...
Models and artifacts saved: rf_final.joblib, xgb_final.joblib, feature_cols.joblib, model_metadata.joblib


In [11]:
# --- Inference: robust prediction + aggregation into submission.csv ---

# load models in case it's a new session
feature_cols = joblib.load('feature_cols.joblib')
metadata = joblib.load('model_metadata.joblib')
rf_model = joblib.load('rf_final.joblib')
xgb_model = joblib.load('xgb_final.joblib')

# Models per axis
rf_per_ch = None
xgb_per_ch = None
if rf_model is None:
    rf_ch_files = sorted([f for f in os.listdir('.') if f.startswith('rf_ch') and f.endswith('.joblib')])
    if rf_ch_files:
        rf_per_ch = [joblib.load(f) for f in rf_ch_files]
        print("RF per axis loaded:", rf_ch_files)
if xgb_model is None:
    xgb_ch_files = sorted([f for f in os.listdir('.') if f.startswith('xgb_ch') and f.endswith('.joblib')])
    if xgb_ch_files:
        xgb_per_ch = [joblib.load(f) for f in xgb_ch_files]
        print("XGB per axis loaded:", xgb_ch_files)

if rf_model is None and rf_per_ch is None:
    raise FileNotFoundError("No RF model found (rf_final.joblib or rf_ch*.joblib).")

# Residuals prediction function
def predict_residuals_for_adapted(feat_df):
    for c in feature_cols:
        if c not in feat_df.columns:
            feat_df[c] = 0.0
    feat_df = feat_df[feature_cols].astype(float)
    Xmat = feat_df.values

    # RF predict
    if rf_model is not None:
        p_rf = rf_model.predict(Xmat)  # (N,3)
    else:
        preds = [m.predict(Xmat) for m in rf_per_ch]
        p_rf = np.column_stack(preds)

    # XGB predict
    if xgb_model is not None:
        p_xg = xgb_model.predict(Xmat)
    elif xgb_per_ch is not None:
        preds = [m.predict(Xmat) for m in xgb_per_ch]
        p_xg = np.column_stack(preds)
    else:
        p_xg = None

    if p_xg is None:
        return p_rf
    return (p_rf + p_xg) / 2.0  # simple average

# iterate test set and aggregate by ID
pred_map = {}  # key: ID -> dict with all columns x_1..x_5,y_1..y_5,z_1..z_5, resname, resid
missing_counts_before = 0
total_rows = 0
start_time = time.time()

for idx, row in test_seqs.iterrows():
    if idx % 10 == 0:
        print(f"Processing {idx} | {time.time()-start_time:.1f}s")
    tid, seq = row['target_id'], row['sequence']
    preds_adapted = predict_rna_structures(row, train_seqs, train_coords_dict, n_predictions=5)
    # if fewer than 5 returned, fill logically with what is available
    n_preds = len(preds_adapted)
    if n_preds == 0:
        # fallback: straight line per chain
        segments = test_segs_map.get(tid, [(0, len(seq))])
        coords = np.zeros((len(seq),3), dtype=float)
        for (s,e) in segments:
            for j in range(s+1,e):
                coords[j] = coords[j-1] + [5.95,0,0]
        preds_adapted = [coords]  # one candidate

    for i_pred, adapted_coords in enumerate(preds_adapted):
        # apply residuals prediction for all residues in this adapted_coords
        L = len(seq)
        feat_rows = []
        W = metadata.get('window', 2)
        for i in range(L):
            f = {}
            f['pos'] = i
            f['pos_norm'] = i / max(1, L-1)
            f['seq_len'] = L
            f['sim_score'] = 0.0
            f['ax'], f['ay'], f['az'] = adapted_coords[i]
            w = seq_window_onehot(seq, i, W=W)
            for j, val in enumerate(w):
                f[f'w_{j}'] = float(val)
            feat_rows.append(f)
        feat_df = pd.DataFrame(feat_rows)
        # ensure column order
        for c in feature_cols:
            if c not in feat_df.columns:
                feat_df[c] = 0.0
        feat_df = feat_df[feature_cols]

        resid_pred = predict_residuals_for_adapted(feat_df)  # (L,3)
        corrected = adapted_coords + resid_pred
        refined = adaptive_rna_constraints(corrected, tid, confidence=0.8, passes=2)

        # fill map by ID
        for j in range(L):
            total_rows += 1
            ID = f"{tid}_{j+1}"
            # create base entry if not exists
            if ID not in pred_map:
                # initialize all columns x_1..x_5,y_1..y_5,z_1..z_5 with NaN (will fill later)
                d = {'ID': ID, 'resname': seq[j], 'resid': j+1}
                for k in range(1,6):
                    d[f'x_{k}'] = np.nan
                    d[f'y_{k}'] = np.nan
                    d[f'z_{k}'] = np.nan
                pred_map[ID] = d
            # set corresponding triplet (i_pred may be >=5 if function generates more; limit to 5)
            col_idx = i_pred + 1
            if col_idx <= 5:
                pred_map[ID][f'x_{col_idx}'] = float(refined[j,0])
                pred_map[ID][f'y_{col_idx}'] = float(refined[j,1])
                pred_map[ID][f'z_{col_idx}'] = float(refined[j,2])

# Convert to DataFrame and sort by ID/resid (optional: keep test_seqs order)
rows = list(pred_map.values())
sub = pd.DataFrame(rows)
# ensure expected columns and order: ID,resname,resid,x_1,y_1,z_1,...,x_5,y_5,z_5
cols = ['ID','resname','resid'] + [f'{c}_{i}' for i in range(1,6) for c in ['x','y','z']]
for c in cols:
    if c not in sub.columns:
        sub[c] = 0.0
sub = sub[cols]

# clip coords and save
coord_cols = [c for c in cols if c.startswith(('x_','y_','z_'))]
sub[coord_cols] = sub[coord_cols].clip(-999.999, 9999.999)

sub.to_csv('submission.csv', index=False)
print("submission.csv saved with", len(sub), "rows.")
sub.head(10)

Processing 0 | 0.0s
Processing 10 | 135.1s
Processing 20 | 142.6s
submission.csv saved with 9762 rows.


,ID,resname,resid,x_1,y_1,z_1,x_2,y_2,z_2,x_3,y_3,z_3,x_4,y_4,z_4,x_5,y_5,z_5
0,8ZNQ_1,A,1,161.694678,182.648592,196.881677,-31.575007,-24.985690,16.865704,6.977852,-40.847913,12.180672,5.124279,-18.471605,41.666795,64.896253,-3.506433,1.282732
1,8ZNQ_2,C,2,157.666965,174.819268,196.019805,-28.665769,-26.994636,21.476258,10.807732,-40.002283,15.922828,11.465611,-16.192393,39.534814,64.726635,-8.409687,6.136341
2,8ZNQ_3,C,3,153.218130,175.983745,199.085249,-26.986273,-28.245894,26.678762,6.884595,-32.006772,12.771456,8.195892,-11.239635,31.342928,57.874354,-10.870275,6.836482
3,8ZNQ_4,G,4,157.710612,178.736468,205.385907,-22.315158,-27.631069,29.425970,10.966890,-27.334953,14.741784,10.855853,-12.775670,29.973608,57.786435,-15.780308,9.059042
4,8ZNQ_5,U,5,158.227952,182.084280,206.835447,-17.525756,-26.580142,21.277124,13.920715,-23.974731,10.068780,13.055820,-15.332476,25.611703,54.226545,-18.795122,6.216053
5,8ZNQ_6,G,6,156.810619,185.981081,209.079932,-13.285221,-25.442847,21.657890,14.471840,-21.845770,4.935187,16.752743,-17.721862,21.286940,54.517795,-22.738465,3.176257
6,8ZNQ_7,A,7,156.981249,190.634683,210.641121,-10.184631,-21.791500,19.668594,9.774395,-38.512517,5.534942,12.466790,-20.530829,20.322148,55.477534,-27.274411,7.541017
7,8ZNQ_8,C,8,153.121369,189.607596,210.502018,-7.754230,-18.768145,16.454905,1.910345,-38.809355,1.175913,7.379107,-22.825547,16.304402,61.432370,-29.321875,7.020902
8,8ZNQ_9,G,9,150.449939,191.055532,209.232872,-12.651575,-13.885579,18.683422,-4.250051,-45.252148,3.053867,5.356645,-28.412325,20.434603,59.600489,-26.452647,5.104496
9,8ZNQ_10,G,10,150.303806,190.821752,207.859128,-16.782862,-8.699739,18.408282,-9.755507,-39.593742,1.286989,0.551812,-24.360974,15.848723,62.361756,-26.294710,3.420751
